In [15]:
import pandas as pd
import json
import numpy as np

## functions

In [75]:
def load_africa_data():
    africa = pd.read_excel('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/04-fakenews-Repository/06-virality/africa/social/0-prepare_data/facebook_africa.xlsx')[['clean_link', 
    'id_post', 
    'id_desinformacion', 
    'Verdict']].rename(columns = {'clean_link':'link_desinformacion'})
    africa['link_post'] = africa.link_desinformacion
    africa['label_desinformacion'] = africa['Verdict'].apply(lambda x: clean_label(x))
    return(africa)

def clean_label(x):
    if x in ['Incorrect','Understated, exaggerated and incorrect',
            'Seven incorrect, three correct, three unproven and one mostly correct']:
        return('fake')
    elif x in ['Mostly correct',
               'A range of verdicts ranging from misleading to unproven and correct',
              'Checked', 'Misleading']:
        return('misleading')
    else:
        return('true')

def load_panel():
    social = pd.read_excel('/Users/cblanesg/misinformation_socialmedia/data/4-panel_data/social/panel_social.xlsx')
    return(social)

In [77]:
len(load_social_data().id_desinformacion.unique())

366

In [30]:
def missing_factchecks(panel):
    panel = panel[['id_desinformacion', 'date_factcheck_facebook_clean']].drop_duplicates()
    panel['missing_factcheck'] = np.where(pd.isna(panel['date_factcheck_facebook_clean']), 1, 0)
    panel_counts = panel.groupby(['missing_factcheck']).count()
    return(panel_counts)

In [63]:
def timeseries(panel):
    ## data of factcheck
    panel_fc = panel[['id_desinformacion', 'date_factcheck_facebook_clean']].drop_duplicates()
    panel_fc['factcheck'] = np.where(pd.isna(panel_fc['date_factcheck_facebook_clean']), 0, 1)
    panel_fc = panel_fc[['factcheck', 'id_desinformacion']]
    
    ## data of time series
    panel = panel[abs(panel['n_days_since_factcheck']) <= 15]
    
    panel = panel[['date_timestep_clean', 'id_desinformacion']]
    panel_counts = panel.groupby(['id_desinformacion']).count().reset_index()
    
    panel_counts['dummy_timeseries'] = np.where(panel_counts['date_timestep_clean'] > 1, 1, 0)
    panel_ts = panel_counts[['id_desinformacion', 'dummy_timeseries']]  

    all_data = pd.merge(left = panel_fc, 
            right = panel_ts, 
            on = 'id_desinformacion', 
            how = 'left')
    
    counts = all_data.groupby(['dummy_timeseries', 'factcheck']).count()
    
    return(counts)




In [65]:
timeseries(social)

,,id_desinformacion
dummy_timeseries,factcheck,
1.0,1,72


In [68]:
    panel = social[abs(social['n_days_since_factcheck']) <= 15]
    panel = panel[['date_timestep_clean', 'id_desinformacion']]
    panel_counts = panel.groupby(['id_desinformacion']).count().reset_index()
    
    panel_counts['dummy_timeseries'] = np.where(panel_counts['date_timestep_clean'] > 1, 1, 0)
    panel_ts = panel_counts[['id_desinformacion', 'dummy_timeseries']]  



In [71]:
panel.groupby(['id_desinformacion']).count().reset_index().date_timestep_clean.unique()

array([17, 18, 13, 16,  6, 21,  8, 19,  3, 20,  9, 15,  7, 12,  5, 11, 14,
        4])

In [74]:
len(panel_counts.id_desinformacion.unique())

72

## Load data

In [18]:
social = load_panel()

In [35]:
social[['date_timestep_clean', 'id_desinformacion']]

,date_timestep_clean,id_desinformacion
0,2021-02-10,ba0b0a04-74ab-4e00-a01b-3ddb80269c20
1,2021-02-09,280bccba-0705-4ff2-ab7c-23009d592f46
2,2020-11-19,8bac89f4-a7e4-493b-92b0-4e396fa9c32b
3,2020-11-21,55337101-2f6b-44ad-a2e9-166a498f7687
4,2021-02-09,214810e0-fe0b-11ea-998d-acde48001122
...,...,...
2608,2020-07-14,a5520168-e9da-3686-ad22-0a1586019645
2609,2020-07-15,a5520168-e9da-3686-ad22-0a1586019645
2610,2020-07-17,a5520168-e9da-3686-ad22-0a1586019645
2611,2020-07-18,a5520168-e9da-3686-ad22-0a1586019645


In [22]:
missing_factchecks(social)

,id_desinformacion,date_factcheck_facebook_clean
missing_factcheck,,
0,215,215
1,124,0


In [29]:
print('Pct of posts with no factcheck: ', 124/(124 + 215)*100, 
      '\nPct of posts with factcheck: ',215/(124 + 215)*100)

Pct of posts with no factcheck:  36.57817109144543 
Pct of posts with factcheck:  63.421828908554566


In [44]:
timeseries(social)

,date_timestep_clean
timeseries,
0,206
1,133


In [45]:
print('Pct of posts with timeseries data: ', 133/(133 + 206)*100, 
      '\nPct of posts with factcheck: ',206/(133 + 206)*100)

Pct of posts with timeseries data:  39.233038348082594 
Pct of posts with factcheck:  60.766961651917406


In [61]:
social.columns

Index(['Unnamed: 0', 'source', 'id_desinformacion', 'id_post', 'id_factcheck',
       'fact_checker', 'date_partnership_facebook', 'poynter_facebook',
       'label_desinformacion', 'no_factcheck', 'no_socialmedia_factcheck',
       'date_post', 'reactions_facebook', 'shares_facebook', 'likes_facebook',
       'comments_facebook', 'date_post_clean', 'date_timestep_clean',
       'date_factcheck_facebook_clean', 'n_days_since_publicacion',
       'n_days_since_factcheck', 'observations_cumsum', 'change_likes',
       'change_comments', 'change_shares', 'change_reactions',
       'growth_rate_likes', 'growth_rate_comments', 'growth_rate_shares',
       'growth_rate_reactions'],
      dtype='object')